In [86]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator




# Función para crear la VGG
def build_vgg(input_shape, num_classes):

  model = models.Sequential()

  # Bloque 1
  model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape))
  model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
  model.add(layers.MaxPooling2D((2, 2)))

  # Bloque 2
  model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
  model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
  model.add(layers.MaxPooling2D((2, 2)))

  # Bloque 3
  model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
  model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
  model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
  model.add(layers.MaxPooling2D((2, 2)))

  # Bloque 4
  model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
  model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
  model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
  model.add(layers.MaxPooling2D((2, 2)))

  # Capas completamente conectadas
  model.add(layers.Flatten())
  model.add(layers.Dense(1024, activation='relu'))
  model.add(layers.Dense(1024, activation='relu'))

  # Capa de salida
  model.add(layers.Dense(1, activation='softmax'))

  return model

In [87]:
# Cargar datos
train_dir =  'C:/Users/paola/OneDrive/Escritorio/histopatologia/train'
val_dir =  'C:/Users/paola/OneDrive/Escritorio/histopatologia/test'
image_size = (24, 24)

In [88]:
# preprocesar datos
train_gen = ImageDataGenerator(rescale=1./24)
valid_gen = ImageDataGenerator(rescale=1./24)

train_generator = train_gen.flow_from_directory(
        train_dir,
        target_size=image_size,
        batch_size=1024)

valid_generator = valid_gen.flow_from_directory(
        val_dir,
        target_size=image_size,
        batch_size=1024)
        

Found 194001 images belonging to 2 classes.
Found 83523 images belonging to 2 classes.


In [92]:
# Crear modelo VGG
input_shape = image_size + (3,)
num_classes = 1

model = build_vgg(input_shape, num_classes)

In [93]:
# Compilar modelo
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_173 (Conv2D)         (None, 24, 24, 64)        1792      
                                                                 
 conv2d_174 (Conv2D)         (None, 24, 24, 64)        36928     
                                                                 
 max_pooling2d_67 (MaxPoolin  (None, 12, 12, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_175 (Conv2D)         (None, 12, 12, 128)       73856     
                                                                 
 conv2d_176 (Conv2D)         (None, 12, 12, 128)       147584    
                                                                 
 max_pooling2d_68 (MaxPoolin  (None, 6, 6, 128)        0         
 g2D)                                                

In [94]:
# Entrenar modelo
model.fit(train_generator, validation_data=valid_generator, epochs=3)

Epoch 1/3
190/190 [==============================] - 4321s 23s/step - loss: 0.6944 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/3
190/190 [==============================] - 4533s 24s/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/3
190/190 [==============================] - 5035s 27s/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000


In [97]:
_, accuracy = model.evaluate(x=valid_generator)
print('Accuracy: %.2f' % (accuracy*100))

82/82 [==============================] - 309s 4s/step - loss: 0.6931 - accuracy: 0.5000
Accuracy: 50.00


In [99]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
import numpy as np

# Predicciones en el conjunto de validación
y_pred = model.predict(valid_generator)
y_true = valid_generator.classes  # Etiquetas verdaderas

# Convertir las probabilidades a etiquetas binarias (0 o 1) usando un umbral
threshold = 0.5
y_pred_binary = (y_pred > threshold).astype(int)

# Calcular métricas
accuracy = accuracy_score(y_true, y_pred_binary)
precision = precision_score(y_true, y_pred_binary)
recall = recall_score(y_true, y_pred_binary)
roc_auc = roc_auc_score(y_true, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'AUC-ROC: {roc_auc}')


82/82 [==============================] - 282s 3s/step
Accuracy: 0.7154077320019635
Precision: 0.7154077320019635
Recall: 1.0
AUC-ROC: 0.5
